In [1]:
import pandas as pd
from datetime import datetime

session = "181"
pos_dir = './out/pos'
data_dir = "./data/{}".format(session)


In [2]:
"""
@author Riordan Dervin Alfredo <riordan.alfredo@monash.edu>
"""
# Run to extract all pozyx position data
# json -> csv

data_in = "{}/{}.json".format(data_dir, session)
sync_path = "{}/sync.txt".format(data_dir)
%run pozyx_extraction.py -p $data_in -o "out/pos/" -s {sync_path}

ModuleNotFoundError: No module named 'simplejson'

In [4]:
colour = "YELLOW"
file_date = "18-Aug-2021_15-33-14-715"

In [5]:
# install it separately https://ffmpeg.org/. Use this to re-sample audio data to 48KHz
# 
audio_in = "{}/simulation_{}_{}_audio.wav".format(data_dir,colour,file_date)
audio_out = "{}/sim_{}.wav".format(data_dir,colour)

print(audio_in)
print(audio_out)

!ffmpeg -i {audio_in} -ar 48000 {audio_out}

./data/181/simulation_YELLOW_18-Aug-2021_15-33-14-715_audio.wav
./data/181/sim_YELLOW.wav


'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
# TODO: replace audio file name!
# A script to convert speech to text, wav -> csv
audio_sim_dir = './out/audio-sim'
audio_csv_out = "{}/{}_{}.csv".format(audio_sim_dir, session, colour)

%run hive_automation.py -a {audio_out} -o {audio_csv_out} -s "145" -w "1" -t "3"

start
here
 before vad
1646.0509791666666
thread_started: speech2text thread 0


  0%|          | 0/24 [00:00<?, ?it/s]

thread_started: speech2text thread 1
thread_started: speech2text thread 2


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

thread_finished: speech2text thread 0
thread_finished: speech2text thread 2
thread_finished: speech2text thread 1
object


In [24]:
# combine position and audio information

dfp = pd.read_csv('{}/{}_{}.csv'.format(pos_dir,session,colour))
dfa = pd.read_csv('{}/{}_{}.csv'.format(audio_sim_dir,session,colour))

dfp.head()

,Unnamed: 0,audio_start_timestamp,audio time,x,y
0,0,1.629265e+09,0:08:34,1852.0,-1691.0
1,1,1.629265e+09,0:08:35,1852.0,-1691.0
2,2,1.629265e+09,0:08:36,1852.0,-1691.0
3,3,1.629265e+09,0:08:37,1852.0,-1691.0
4,4,1.629265e+09,0:08:38,1852.0,-1691.0


In [25]:
res = pd.merge(dfp, dfa, on="audio time")
final = res.drop(labels=["Unnamed: 0_x", "Unnamed: 0_y", "session"], axis=1)
final['tagId'] = colour
final.head()

,audio_start_timestamp,audio time,x,y,audio,tagId
0,1.629265e+09,0:08:34,1852.0,-1691.0,0,YELLOW
1,1.629265e+09,0:08:35,1852.0,-1691.0,0,YELLOW
2,1.629265e+09,0:08:36,1852.0,-1691.0,0,YELLOW
3,1.629265e+09,0:08:37,1852.0,-1691.0,0,YELLOW
4,1.629265e+09,0:08:38,1852.0,-1691.0,0,YELLOW


In [26]:

result_dir =  "./out/result"
result_csv = "{}/{}_{}.csv".format(result_dir,session,colour)
final.to_csv(result_csv, sep=',', encoding='utf-8', index=False)

# Save to front-end too!
# final.to_csv("{}/{}_{}.csv".format(client_dir,session,colour), sep=',', encoding='utf-8', index=False)

# Before you continue...
Please repeat all available colours before combining them in the following code cells

In [27]:

# combine all data
df = pd.concat(map(pd.read_csv, [
    '{}/{}_RED.csv'.format(result_dir, session), 
    '{}/{}_YELLOW.csv'.format(result_dir, session), 
    '{}/{}_BLUE.csv'.format(result_dir, session),
    '{}/{}_GREEN.csv'.format(result_dir, session)]), ignore_index=True)

df = df.sort_values(by='audio time')

In [28]:
client_dir = "../client/src/projects/hive/data"
df.to_csv("{}/{}_all.csv".format(client_dir,session), sep=',', encoding='utf-8', index=False)

df

,audio_start_timestamp,audio time,x,y,audio,tagId
0,1.629265e+09,0:00:01,-524.789615,4147.306944,0,RED
2779,1.629265e+09,0:00:02,-167.668741,3779.885582,0,BLUE
1,1.629265e+09,0:00:02,-326.150993,3904.732677,0,RED
2,1.629265e+09,0:00:03,336.294696,3196.755684,0,RED
2780,1.629265e+09,0:00:03,843.721810,3173.210523,0,BLUE
...,...,...,...,...,...,...
1644,1.629265e+09,0:27:25,73.828179,7465.051546,0,RED
1645,1.629265e+09,0:27:26,63.101696,7445.032204,0,RED
4423,1.629265e+09,0:27:26,4107.109982,2110.458546,0,BLUE
2778,1.629265e+09,0:27:26,5677.768964,883.438448,0,YELLOW
